In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
housing_path ='../input/california-housing-prices/housing.csv'
housing_data = pd.read_csv(housing_path)

In [ ]:
housing_data.head()

In [ ]:
housing_data.info()

In [ ]:
housing_data.hist(bins=50, figsize=(20,15)) 
plt.show()

In [ ]:
housing_data.columns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_set, test_set = train_test_split(housing_data, test_size=0.2, random_state=42)

In [ ]:
housing_data["income_cat"] = np.ceil(housing_data["median_income"] / 1.5) 
housing_data["income_cat"].where(housing_data["income_cat"] < 5, 5.0, inplace=True)
housing_data["income_cat"].value_counts()

In [ ]:
housing_data.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)
plt.show()

In [ ]:
housing_data.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, 
                  s=housing_data["population"]/100, label="population", 
                  c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,) 
plt.legend()
plt.show()

In [ ]:
corr_matrix = housing_data.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing_data["rooms_per_household"] = housing_data["total_rooms"]/housing_data["households"] 
housing_data["bedrooms_per_room"] = housing_data["total_bedrooms"]/housing_data["total_rooms"] 
housing_data["population_per_household"] = housing_data["population"]/housing_data["households"]

In [ ]:
corr_matrix = housing_data.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 

for train_index, test_index in split.split(housing_data, housing_data["income_cat"]):
    strat_train_set = housing_data.loc[train_index] 
    strat_test_set = housing_data.loc[test_index]

In [ ]:

housing = strat_train_set.drop("median_house_value", axis=1) 
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
strat_train_set.columns

In [ ]:
housing.columns

In [ ]:
median = housing["total_bedrooms"].median() 
housing["total_bedrooms"].fillna(median)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
imputer.statistics_

In [ ]:
 housing_num.median().values

In [ ]:
housing_num.info()

In [ ]:
X = imputer.transform(housing_num)


In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
set(housing_cat_encoded)

print(encoder.classes_)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1)) 
housing_cat_1hot

In [ ]:
housing_cat_1hot.toarray()